# Report Basics of Mobile Robotics

Report of Jean, Vadim, Sara and Lisa.

**Table of Contents:**
1. [Introduction](#Introduction)
2. [Vision](#Vision)
3. [Global Path Planning](#Global-Path-Planning)
4. [Localization with Kalman Filter](#Kalman-Filter)
5. [Motor Control](#Motor-Control)
6. [Local Obstacle Avoidance](#Local-Obstacle-Avoidance)
7. [Putting it all together](#Putting-it-all-together)
8. [Conclusion](#Conclusion)

## Introduction

<span style="color:red">
Describe our general project idea. Introduction to the environement and choices we made. General function of the main components.

Structure of the code
</span>

## Vision

<span style="color:red">
Code required to execute vision module independently. Don't describe the code but explain what is behind it (i.e. theory, algorithms, measurements, choice of parameters, etc.).

CITE YOUR SOURCES
</span>



Let's start by defining what main functionalities the vision module should accomplish.
1. Find the **borders of the map** in the image.
2. Detect the **global obstacles**, which are placed on the map.
3. Locate the **position of the goal** in the map.
4. Locate the **position of the robot** in the map.

As our camera has fixed position points 1,2 and 4 only need to be done once during the initalization of the map, while the localization of the robot will be used during runtime as a measurement for the Kalman filter.

Additionally a visualition during runtime of all the above points, as well as the estimated robot position given by the filter and the planned path will ease the development further.

All the code used for the vision module can be found under `src/vision`

### Borders of the map

To find the borders of the map, we decided to use [aruco markers](https://www.uco.es/investiga/grupos/ava/portfolio/aruco/l) - a robust library available for open cv to detect predefined markers in an image used for pose estimation - to mark the corners of our map.

For this we find the first corner of each four corner markers (markers have ids 1-4). We then apply a perspective transform where we map the four given corners to predefined image of size 350x240 pixel. The choice of the cropped map size is based on the actual size of our image, which is 1050mmx720mm, so every pixel in our cropped map corresponds to 3 mm in reality.

The principle can be seen on this image:

<img src="./media/perspective_transform.png" alt="Perspective Transform" width="500"/>

The Perspective transform allows us to rectify the map in our image. As the camera looks at the scene with a certain angle, the map appears distorted on the image. The perspective transform projects the map again onto a regular rectangle. It though does not correct for any radial distortion of the camera itself. For this the camera would need to be calibrated but since the precision without this step was sufficient for our purposes, we skipped this step.

This whole process is done in the [camera class](src/vision/camera.py) in `src/vision/camera.py::_find_corners()`, the for illustration purposes required code is copied below.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from src.vision.helpers import get_corner, perspective_transform, Hyperparameters, read_yaml

example_img_path = "media/example_map.jpg"
frame = cv2.imread(example_img_path)
global_corners = [None, None, None, None]
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
detector = cv2.aruco.ArucoDetector(aruco_dict, cv2.aruco.DetectorParameters())
hyperparams = Hyperparameters(read_yaml())

# This function is taken from src/vision/camera.py and modified to make it work without
# the class context.
def _find_corners(frame, aruco_detecter, show=False):
    """
    Detects the aruco markers of the corners and updates the position

    Args:
        show (bool): If True shows the frame with drawn markers. Default: False
    """
    if show:
        img = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    corners, ids, rejected = aruco_detecter.detectMarkers(gray)
    if ids is not None:
        # the 4 corner of our map have ids 1,2,3,4
        for i, id in enumerate(ids):
            if id >= 1 and id <= 4:  # only consider corner markers
                global_corners[id[0]-1] = get_corner(corners[i])
        if show:
            cv2.aruco.drawDetectedMarkers(img, corners, ids)
    
    if show:
        return img
    
    
fig, axes = plt.subplots(2, 2, figsize=(8, 6), sharey='row')  
axes[0, 0].set_title("Raw Frame")
axes[0, 0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title("Detected Aruco markers in the image")
markers = _find_corners(frame, detector, show=True)
axes[0, 1].imshow(cv2.cvtColor(markers, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title("Transformed image")
warped = perspective_transform(frame, global_corners, hyperparams.map_size[0], hyperparams.map_size[1])
axes[1, 0].imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))
axes[1, 1].axis('off')
plt.tight_layout()
plt.show()

### Global obstacles

The global (black) obstacles are extracted from the imag of the map by:
1. thresholding the image to extract the black pixels
2. using a canny edge detector to find the edges
3. finding the contours and filtering them by area

**Thresholding**:

Thresholding is the simplest method of image segmentation where every pixel value above, below or within certain thresholds is set to a certain value. As our obstaces are black, we threshold on the BGR color space and set every pixel with a value below a certain threshold to black. The extracted obstacles appear now as white on a binary image. 
The thresholding step has three hyperparameters, one for each color channel.

__Canny Edge Detection__:

The Canny edge detector is an algorithm to detect various types of edges in an image. We used it to detect the edges of the binary image of the obstacles.

__Contours__:

From the edges we can find the contour of the obstacle, which is a set of boundary points of the obstacle. Since the _findContours_ function in opencv is sensitive to non-closed contours, we use a morphology operation to close the contours before applying the function. This step significantly improved the robustness of the contour detection.
To filter for noise we only keep contours with an area above a certain threshold.

The function `src/vision/camera.py::_extract_obstacles()` contains the code reuiqred for this step and saves a map with the detected obstacles drawn on it.

In [ ]:
def _extract_obstacles(cropped_img, hyperparams: Hyperparameters, show_warped: bool = False):
    """
    Thresholds the current frame to see the black obstacles.

    Args:
        show_warped (bool): If True shows the cropped image with drawn contours. 
                            Default: False.
    """
    warped = cropped_img.copy()
    thresholded = cv2.inRange(warped, (0, 0, 0), 
                                (hyperparams.obstacles.blue, 
                                hyperparams.obstacles.green,
                                hyperparams.obstacles.red))
    
    canny = cv2.Canny(thresholded, 94, 98, apertureSize=3)
    kernel = np.ones((hyperparams.obstacles.kernel_size, 
                      hyperparams.obstacles.kernel_size), np.uint8)
    morph = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel, iterations=2)

    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # filter controus by area
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) > hyperparams.obstacles.area]
    map = np.zeros_like(warped, dtype=np.uint8)
    
    cv2.drawContours(map, contours, -1, (255, 255, 255), cv2.FILLED)
    if show_warped:
        cv2.drawContours(warped, contours, -1, (0, 0, 255), 1)
        return warped, map
    return map

fig, axes = plt.subplots(1, 3, figsize=(8, 6), sharey='row')

axes[0].set_title("Cropped Frame")
axes[0].imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))

axes[1].set_title("Detected Contours")
contours, map = _extract_obstacles(warped, hyperparams, show_warped=True)
axes[1].imshow(cv2.cvtColor(contours, cv2.COLOR_BGR2RGB))

axes[2].set_title("Binary Obstacle Map")
axes[2].imshow(cv2.cvtColor(map, cv2.COLOR_BGR2RGB))

plt.tight_layout()
plt.show()

### Goal Position

To find the goal in the map we use a similar approach as before with different thresholds for BGR color space. As our goal is red we want the red channel to be above a certain value and the blue and green channel to be below a certain value. The goal is then the center of the contour in the image.



In [ ]:
def _extract_goal(cropped_img, hyperparams: Hyperparameters, show_warped: bool = False):
        """
        Thresholds the current frame to extract the goal position.

        Args:
            show_warped (bool): If True shows the cropped image with drawn contour. 
                                Default: False.
        """
        warped = cropped_img.copy()
        thresholded = cv2.inRange(warped, (0, 0, hyperparams.goal.red), 
                                    (hyperparams.goal.blue, 
                                     hyperparams.goal.green,255))
        
        canny = cv2.Canny(thresholded, 94, 98, apertureSize=3)
        kernel = np.ones((hyperparams.goal.kernel_size, 
                          hyperparams.goal.kernel_size), np.uint8)
        morph = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel, iterations=2)

        contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # filter controus where area < 500
        contours = [cnt for cnt in contours if cv2.contourArea(cnt) > hyperparams.goal.area]
        if contours:
            M = cv2.moments(contours[0])
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            goal_position = [cx, cy]
        
        if show_warped:
            cv2.drawContours(warped, contours, -1, (0, 0, 0), 2)
            cv2.circle(warped, goal_position, 5, (255, 0, 0), cv2.FILLED)
            return warped, goal_position
 
contours, goal = _extract_goal(warped, hyperparams, show_warped=True)
print(f"Goal position: {goal}")

fig, axes = plt.subplots(1, 2, figsize=(8, 6), sharey='row')

axes[0].set_title("Cropped Frame")
axes[0].imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))

axes[1].set_title("Detected Goal Position")
axes[1].imshow(cv2.cvtColor(contours, cv2.COLOR_BGR2RGB))

plt.tight_layout()
plt.show()

# TODO: Get better example image with goood thresholds


### Robot Position

The robot position is found by detection the arcuo marker with id 42 which is taped on top of the robot. The postion of the robot is the center of the marker, whereas the orientation is the angle formed between the marker and the x-axis. 

The center and orientation can be calculated from the four corners of the markers (given by the aruco library) as follows:

<img src="./media/robot_pose.png" alt="Robot Pose Calculation" width="600"/>

As the aruco marker is not detected in every frame and the orientation is influenced by noise, we use a moving average filter to smoothen the results. In case the the robot is not detected for several consecutive frames, the function returns `np.nan` for the position and orientation. This serves as a flag for the Kalman filter to ignore the measurement in case the camera is hidden.
The specific code can be found [here](src/vision/measurements.py).

The function `src/vision/camera.py::_extract_robot_pose()` contains the code required for this step.

In [ ]:
from src.vision.measurements import Orientation, Position
def _extract_robot_pose(cropped_img, aruco_detector, robot_orientation: Orientation, 
                        robot_position: Position, show: bool = False):
    """
    Extracts the robot pose from latest frame
    Args:
        show (bool): If True shows the found aruco marker
    """
    warped = cropped_img.copy() 
    if show:
        img = warped.copy()
    gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)

    corners, ids, rejected = aruco_detector.detectMarkers(gray)
    if ids is not None:
        # the 4 corner of our map have ids 1,2,3,4
        for i, id in enumerate(ids):
            if id == 42:  # only consider corner markers
                c1 = corners[0][0][0].astype(int)
                c2 = corners[0][0][1].astype(int)
                c3 = corners[0][0][2].astype(int)
                center = np.mean([c1, c3], axis=0)
                p = np.mean([c1, c2], axis=0)
                # angle = np.arctan2((center-p)[1], (p-center)[0])
                robot_orientation.update(center-p)
                robot_position.update(center)
            if show:
                cv2.aruco.drawDetectedMarkers(img, corners, ids)
    else:
        # if no aruco marker found update with nan
        robot_position.update([np.nan, np.nan])
        robot_orientation.update([np.nan, np.nan])
        
    if show:
        if not np.isnan(robot_orientation.value):
            cv2.putText(img, str(np.degrees(robot_orientation.value).astype(int)), (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))
        return img
    
robot_orientation = Orientation()
robot_position = Position()
robot_pose = _extract_robot_pose(warped, detector, robot_orientation, robot_position, show=True)
print(f"Robot position: {robot_position.value}, Robot orientation: {np.degrees(robot_orientation.value):.2f}°")

fig, axes = plt.subplots(1, 2, figsize=(8, 6), sharey='row')

axes[0].set_title("Cropped Frame")
axes[0].imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))

axes[1].set_title("Detected Robot Pose and Orientation")
axes[1].imshow(cv2.cvtColor(robot_pose, cv2.COLOR_BGR2RGB))

plt.tight_layout()
plt.show()

### Visualization

The class camera provides a function `src/vision/camera.py::visualize_map()` which draws the map with all the detected elements on it, as well as the estimated robot position from the kalman filter and the optimal path if inialized. The final visualization can be seen in the gif below.

<img src="./media/example_visualization.gif" alt="Vision Visualization" width="300"/>

### A little word about hyperparameters

Vision depends strongly on the lightning conditions, which change from place to place but also during the day. To ease the tuning for those hyperparameters we created a small script `scripts/threshold_finder.py` which allows to find the correct combination of parameters with live feedback from the camera. The thresholds found are then stored in [src/vision/values](src/vision/values.yml) and loaded during runtime of the main program. 

A screenshot of the script can be seen below:

<p align="top">
    <img src="media/threshold_finder.png" alt="Threshold Finder" width="600"/> 
    <img src="media/threshold_finder_settings.png" alt="Threshold Finder Settings" width="400"/>
</p>


## Global Path Planning

Code required to execute vision module independently. Don't describe the code but explain what is behind it (i.e. theory, algorithms, measurements, choice of parameters, etc.).

CITE YOUR SOURCES

After the extracting the binary matrix of navigable terrain, as well as the positions of the robot and the target, we inflate the obstacles (explain the algo) by the size equal to the maximum distance between the robot's center of rotation and its side. This ensures that for all possible poses the robot does not collide with obstacles.

[code: obstacle inflation]
[plot: inflated obstacles] 


After the matrix is calculated, we use the A-star algorithm to find the shortest path between the the robot and target positions. (optional: intermedaite checkpoint elimination by Sara).

We then delete all the redundant points from the checkpoint list using a "visibility algorithm" (elaborate further). NOTE: why A-star before visibility and not the other way around? Because we don't assume polygonal obstacles so it's not necessarily more robust nor efficient in computation. In the end we are left with a path optimized in terms of both distance and number of checkpoints: 

[code: visibility algo]
[plot: final checkpoints list]
[plot: final checkpoints list over camera image]





## Kalman Filter

# Extended Kalman Filter for Robot Localization

This report provides an explanation of the implementation of the Extended Kalman Filter (EKF) used for robot localization. The EKF estimates the robot's position, orientation, and motion parameters based on sensor inputs (camera data) and motor control signals.

## Introduction to Kalman Filtering
The Kalman Filter is a recursive estimation algorithm used to infer the state of a dynamic system. It is particularly effective for systems with noisy measurements and process models. The **Extended Kalman Filter (EKF)** is a nonlinear extension of the Kalman Filter, suitable for systems where the dynamics or measurement models are nonlinear.

In this implementation, the EKF is used to estimate the robot's state vector:

- **Position**: \((x, y)\) coordinates in the environment
- **Orientation**: Angle \(\theta\) (yaw, in radians)
- **Velocity**: Linear velocity \(v\)
- **Angular velocity**: Rate of change of \(\theta\)

CITE YOUR SOURCES
</span>

## Motor Control

<span style="color:red">
Code required to execute vision module independently. Don't describe the code but explain what is behind it (i.e. theory, algorithms, measurements, choice of parameters, etc.).

CITE YOUR SOURCES
</span>

## Local Obstacle Avoidance

<span style="color:red">
Code required to execute vision module independently. Don't describe the code but explain what is behind it (i.e. theory, algorithms, measurements, choice of parameters, etc.).

CITE YOUR SOURCES
</span>

## Putting it all together

<span style="color:red">
A section which is used to run the overall project and where we can see the path chosen, where the system believes the robot is along the path before and after filtering etc… 

REFERENCE OT THE MAIN FILE.
</span>

## Conclusion

<span style="color:red">
What we learned, what we would do differently, what we would like to add, etc.
</span>

## References:

(link documentation of used python functions that are not immediately clear)